### 패키지 불러오기

In [1]:
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

# ML
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score,precision_score,recall_score, roc_curve, classification_report,precision_recall_curve
from sklearn.metrics import make_scorer
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

### 코랩이용시(드라이브 연동)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
d_path = '/content/drive/MyDrive/Colab Notebooks/Dacon/'
X_train = pd.read_csv(d_path + 'train_features.csv')
y_train = pd.read_csv(d_path + 'train_labels.csv')
X_test = pd.read_csv(d_path + 'test_features.csv')
submission = pd.read_csv(d_path + 'sample_submission.csv')

display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629


,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043


### 쥬피터 이용시(파일경로)

In [ ]:
# X_train = pd.read_csv('train_features.csv')
# y_train = pd.read_csv( 'train_labels.csv')
# X_test = pd.read_csv('test_features.csv')
# submission = pd.read_csv('sample_submission.csv')

----------------------------

### X_pivot_train, X_pivot_test 생성
- X_train, X_test의 데이터를 통해서 id와 time을 제외한 피쳐를 대상으로 id를 기준으로 잡아 합, 평균, 중앙값, 최소값, 최대값, 표준편차, 분산을 구함

In [4]:
X_pivot_train = pd.pivot_table(data = X_train,                   # X_train의 데이터를 통해서
                               values = X_train.columns[2:],     # id와 time을 제외한 피쳐를 대상으로
                               index = 'id',                     # id를 기준으로 잡아
                               aggfunc = ['sum','mean',          # 합, 평균
                                          'median','min','max',  # 중앙값 최소값, 최대값
                                          'std','var'            # 표준편차, 분산 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = X_test, 
                               values = X_test.columns[2:], 
                               index = 'id', 
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

display(X_pivot_train.head(3))
display(X_pivot_test.head(3))

sum                          ...          var                           
         acc_x       acc_y       acc_z  ...         gy_x          gy_y         gy_z
id                                      ...                                        
0   558.797337 -131.082711 -222.252919  ...   176.470384    590.513292   638.834979
1  -459.948117 -190.354639   -2.534051  ...  6279.700472   9217.015511  5707.098884
2    23.901616  -49.441742  375.607013  ...   646.325142  14150.683677   193.775778

[3 rows x 42 columns]

sum                          ...          var                          
           acc_x       acc_y       acc_z  ...         gy_x         gy_y         gy_z
id                                        ...                                       
3125 -611.238360  -11.744605 -139.355669  ...   166.357553  1023.553453   150.102867
3126 -313.705824  367.296809  -42.655405  ...  2665.150566  2089.066820  3795.159662
3127  304.167948  542.291164  -84.658968  ...   518.511372   181.383940   530.909012

[3 rows x 42 columns]

In [5]:
# 생성된 데이터를 확인

X_pivot_train.columns[:5]

MultiIndex([('sum', 'acc_x'),
            ('sum', 'acc_y'),
            ('sum', 'acc_z'),
            ('sum',  'gy_x'),
            ('sum',  'gy_y')],
           )

### X_pivot_train, X_pivot_test 데이터프레임 재구성
- Multiindex 에서 단일 index로 변경


In [6]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
display(X_pivot_train.head())

,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


In [7]:
# index 초기화

X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

display(X_pivot_train.head())
display(X_pivot_test.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,-3.359506,1.182107,0.956149,-0.240638,-0.346749,-1.273569,-2.362230,1.913286,0.591940,-0.624113,-0.786336,-46.254836,-85.887677,-79.930029,1.344268,0.176871,-0.054876,31.644123,69.847244,55.953827,0.191479,0.177131,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,1.740475,1.393294,-0.805767,-0.228905,-0.034583,3.810650,8.043707,-0.655819,-2.156208,-1.295598,-1.019531,-325.328531,-315.096003,-270.980823,1.234020,0.700065,0.888661,286.624363,389.608060,340.170199,0.495528,0.336415,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,0.597877,3.053291,0.140667,-0.062598,0.634781,-8.112557,19.306132,3.568888,-1.142847,-0.690990,0.073846,-164.779067,-249.953944,-44.192071,1.219836,0.650645,1.332992,73.525082,297.320834,55.642836,0.711972,0.147127,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,4.800931,-5.869898,-0.880343,-0.054577,0.231537,8.229938,1.783260,-3.853078,-1.417751,-0.540827,-0.257124,-69.419166,-82.537304,-85.600536,-0.622250,0.283721,0.598814,192.765368,159.083788,56.456908,0.130899,0.194008,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,9.651713,4.453382,-0.941146,-0.168467,0.293556,-1.292194,0.977772,-0.750283,-2.429109,-2.055076,-1.250483,-769.076518,-243.909948,-270.581913,0.599720,1.724782,2.678034,613.972600,284.952954,221.015193,0.495170,0.570305,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,mean_gy_y,mean_gy_z,median_acc_x,median_acc_y,median_acc_z,median_gy_x,median_gy_y,median_gy_z,min_acc_x,min_acc_y,min_acc_z,min_gy_x,min_gy_y,min_gy_z,max_acc_x,max_acc_y,max_acc_z,max_gy_x,max_gy_y,max_gy_z,std_acc_x,std_acc_y,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,3125,-611.238360,-11.744605,-139.355669,-1911.076959,1639.123438,-1200.410049,-1.018731,-0.019574,-0.232259,-3.185128,2.731872,-2.000683,-1.064222,-0.005735,-0.268442,-3.770150,0.108956,-1.607847,-1.564000,-0.470937,-0.573836,-50.429364,-81.607713,-35.446915,-0.275446,0.228040,0.286182,56.953059,96.185341,49.981455,0.236232,0.091641,0.174672,12.897967,31.993022,12.251648,0.055806,0.008398,0.030510,166.357553,1023.553453,150.102867
1,3126,-313.705824,367.296809,-42.655405,-10644.915365,4184.863263,-2162.747150,-0.522843,0.612161,-0.071092,-17.741526,6.974772,-3.604579,-0.677411,0.606215,-0.026089,-14.305258,-0.974696,-10.833508,-1.929033,-0.200678,-1.212052,-273.572486,-97.100707,-147.597574,0.627571,1.708743,0.671876,132.830402,241.240196,169.417650,0.539688,0.333015,0.383800,51.625096,45.706311,61.604867,0.291264,0.110899,0.147302,2665.150566,2089.066820,3795.159662
2,3127,304.167948,542.291164,-84.658968,-1307.846921,-1350.871152,-235.904841,0.506947,0.903819,-0.141098,-2.179745,-2.251452,-0.393175,0.497370,0.931239,-0.156647,0.476247,-2.319172,-3.087735,-0.792916,0.219008,-0.484614,-99.799971,-98.420987,-154.477074,2.972063,1.941820,0.644154,160.426058,74.530763,97.211730,0.219934,0.191485,0.152077,22.770845,13.467885,23.041463,0.048371,0.036666,0.023128,518.511372,181.383940,530.909012
3,3128,-346.561617,-366.333946,18.891749,485.147442,-1790.981310,-14.590798,-0.577603,-0.610557,0.031486,0.808579,-2.984969,-0.024318,-0.880541,-0.507927,-0.092860,1.457625,-0.269600,-0.404583,-1.045889,-1.294482,-0.469924,-229.072919,-168.031080,-117.297766,0.337281,-0.258476,0.702574,119.527887,118.268797,167.860762,0.431713,0.233601,0.326569,42.818157,45.069932,37.967372,0.186376,0.054569,0.106647,1833.394532,2031.298793,1441.521366
4,3129,-443.184021,109.521180,240.781103,-858.922755,865.419381,3447.298941,-0.738640,0.182535,0.401302,-1.431538,1.442366,5.745498,-0.703842,0.122280,0.432678,-3.066063,1.631638,1.866352,-2.153047,-0.860883,-0.631258,-345.447240,-223.475411,-125.598600,0.015642,1.562602,1.037876,366.167357,226.728939,138.130133,0.305797,0.314294,0.261848,92.301963,67.911174,43.353007,0.093512,0.098781,0.068565,8519.652350,4611.927587,1879.483194


In [8]:
# train 데이터 셋 확인

X_pivot_train.shape

(3125, 43)

### 학습에 사용할 데이터(ftr, target)

In [9]:
ftr = X_pivot_train
target = y_train['label']

### 시각화를 위해 데이터를 X_train, X_val, y_train, y_val로 나눔

In [17]:
X_train, X_val, y_train, y_val = train_test_split(ftr, target, test_size=0.2, random_state=156)

In [18]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[('rfc', rfc), ('lgb', lgb), ('xgb', xgb)], voting='soft')

vclf.fit(X_train, y_train)
pred = vclf.predict(X_val)
print('정확도: ', accuracy_score(y_val, pred))

모든 예측을 0으로 했을 때의 정확도:  0.7792


In [19]:
pred_proba = vclf.predict_proba(X_val)

In [42]:
pred_proba

array([[2.00604985e-03, 1.86261835e-01, 7.97030014e-04, ...,
        6.94677351e-04, 2.37385914e-03, 4.96471889e-03],
       [5.31210541e-03, 3.44029723e-04, 1.46740762e-03, ...,
        5.06252225e-05, 5.29542623e-05, 1.46552546e-03],
       [2.91162022e-03, 5.72658525e-01, 2.16923748e-04, ...,
        1.11214135e-04, 3.76465893e-03, 8.95100240e-03],
       ...,
       [4.88258822e-04, 3.16197768e-05, 2.12491755e-05, ...,
        3.71896901e-05, 1.87484545e-05, 6.22183144e-04],
       [1.13550004e-03, 3.70151421e-04, 1.27711912e-03, ...,
        5.55851682e-03, 6.66416305e-04, 2.32013378e-01],
       [2.45420148e-05, 5.24553010e-03, 2.36551606e-05, ...,
        1.90722791e-05, 2.29875999e-03, 2.61811487e-05]])

In [37]:
# 사이킷런의 정확도, 정밀도, 재현율, 오차행렬을 계산하는 API 호출
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# 호출한 지표들을 한꺼번에 계산하는 함수 정의
def get_clf_eval(y_val, pred):
    confusion = confusion_matrix(y_val, pred)
    accuracy = accuracy_score(y_val, pred)
    precision = precision_score(y_val, pred, average='micro')
    recall = recall_score(y_val, pred, average='micro')
    print('오차행렬')
    print(confusion)
    print('정확도 : {:.4f}\n정밀도 : {:.4f}\n재현율 : {:.4f}'.format(accuracy, precision, recall))

In [51]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import precision_recall_curve
%matplotlib inline

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_val, pred)
    accuracy = accuracy_score(y_val, pred)
    precision = precision_score(y_val, pred, average='micro')
    recall = recall_score(y_val, pred, average='micro')
    # F1 스코어 추가
    f1 = f1_score(y_val, pred, average='micro')
    print('오차행렬')
    print(confusion)
    # F1 score print 추가
    print('\n정확도: {:.4f}\n정밀도: {:.4f}\n재현율: {:.4f}\nF1: {:.4f}'.format(accuracy, precision, recall, f1))
    
thresholds = [0.4, 0.45, 0.5, 0.55, 0.6]
pred_proba = vclf.predict_proba(X_val)
get_eval_by_threshold(y_val, pred_proba[:,1].reshape(-1, 1), thresholds)


임계값:  0.4
오차행렬
[[ 1  0  0 ...  0  0  0]
 [ 2  2  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 3  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 [11  0  0 ...  0  0  0]]

정확도: 0.0048
정밀도: 0.0048
재현율: 0.0048
F1: 0.0048

임계값:  0.45
오차행렬
[[ 1  0  0 ...  0  0  0]
 [ 2  2  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 3  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 [11  0  0 ...  0  0  0]]

정확도: 0.0048
정밀도: 0.0048
재현율: 0.0048
F1: 0.0048

임계값:  0.5
오차행렬
[[ 1  0  0 ...  0  0  0]
 [ 2  2  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 3  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 [11  0  0 ...  0  0  0]]

정확도: 0.0048
정밀도: 0.0048
재현율: 0.0048
F1: 0.0048

임계값:  0.55
오차행렬
[[ 1  0  0 ...  0  0  0]
 [ 2  2  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 3  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 [11  0  0 ...  0  0  0]]

정확도: 0.0048
정밀도: 0.0048
재현율: 0.0048
F1: 0.0048

임계값:  0.6
오차행렬
[[ 1  0  0 ...  0  0  0]
 [ 3  1  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 3  0  0 ...  0  0  

In [52]:
# ROC 곡선의 시각화
def roc_curve_plot(y_test, pred_proba_c1):
    #임계값에 따른 FPR, TPR 값을반환 받음
    fprs, tprs, thresholds  = roc_curve(y_val, pred_proba_c1)
    # ROC곡선을 그래프로 그림
    plt.plot(fprs, tprs, label='ROC')
    # 가운데 대각선 직선을 그림
    plt.plot([0,1], [0,1], 'k--', label='Random')
    
    # FPR X축의 Scale을 0.1 단위로 변경, X, Y축 명 설정 등
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1), 2))
    plt.xlim(0, 1)
    plt.ylim(0, 1)
    plt.xlabel('FPR(1-Sensitivity)')
    plt.ylabel('TPR(Recall)')
    plt.legend()
    
roc_curve_plot(y_val, pred_proba[:, 1])

ValueError: ignored

### LGBM(단일모델)

In [ ]:
lgbm_wrapper = LGBMClassifier(n_estimators=1000, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30)

evals = [(ftr, target)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_pivot_test)
pred_proba = lgbm_wrapper.predict_proba(X_pivot_test)[:,:]

[1]	valid_0's multi_logloss: 2.61205	valid_0's multi_logloss: 2.61205
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's multi_logloss: 2.49809	valid_0's multi_logloss: 2.49809
[3]	valid_0's multi_logloss: 2.40317	valid_0's multi_logloss: 2.40317
[4]	valid_0's multi_logloss: 2.31908	valid_0's multi_logloss: 2.31908
[5]	valid_0's multi_logloss: 2.24428	valid_0's multi_logloss: 2.24428
[6]	valid_0's multi_logloss: 2.17546	valid_0's multi_logloss: 2.17546
[7]	valid_0's multi_logloss: 2.11406	valid_0's multi_logloss: 2.11406
[8]	valid_0's multi_logloss: 2.05536	valid_0's multi_logloss: 2.05536
[9]	valid_0's multi_logloss: 1.99797	valid_0's multi_logloss: 1.99797
[10]	valid_0's multi_logloss: 1.9449	valid_0's multi_logloss: 1.9449
[11]	valid_0's multi_logloss: 1.89397	valid_0's multi_logloss: 1.89397
[12]	valid_0's multi_logloss: 1.84579	valid_0's multi_logloss: 1.84579
[13]	valid_0's multi_logloss: 1.80107	valid_0's multi_logloss: 1.80107
[14]	valid_0's multi_logl

In [ ]:
print(pred_proba.shape)

### 앙상블 Voting을 위한 작업

---------------------------------------
### RandomForest 최적 Parameter 찾기

In [ ]:
def SCORES(y_val, pred, proba) :
    #acc = accuracy_score(y_val, pred)
    f1 = f1_score(y_val, pred, average='macro')
    #print(y_val[:10])
    #print(proba[:10])
    auc = roc_auc_score(y_val, proba, average='macro', multi_class='ovo')

In [ ]:
print(y_val.shape)

print(preds.shape)

(625,)
(782,)


In [11]:
rf_model = RandomForestClassifier(n_jobs=-1, random_state=0, min_samples_leaf=1, n_estimators=1200, max_depth=19)

In [15]:
my_hyper_param = {
    "n_estimators"      :[1200],    # 1200 
    "max_depth"         :[19,20],   # 19
    "min_samples_leaf"  :[1,3]      # 1
}

In [16]:
from sklearn.metrics import make_scorer

gcv_model = GridSearchCV(rf_model, param_grid=my_hyper_param, scoring='roc_auc_ovr', refit=True, cv=5, verbose=0)

# scoring=make_scorer(recall_score, average='micro')

gcv_model.fit(ftr, target)

print("best_estimator:", gcv_model.best_estimator_)
print("best_params:",    gcv_model.best_params_)
print("best_score:" ,    gcv_model.best_score_)

KeyboardInterrupt: ignored

### RandomForest 학습

In [12]:
rfc = rf_model
rfc.fit(ftr, target)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=19, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1200,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

### XGBOOST 최적 Parameter 찾기

In [ ]:
estimator = XGBClassifier()
 
parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.05, 0.1],      #so called `eta` value 0.05
              'max_depth': [6,8,10],             # 6
              'n_estimators': [300,350,400],     # 350 
              'seed': [1337]}

clf = GridSearchCV(estimator, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='roc_auc_ovr',
                   verbose=2, refit=True)
 
clf.fit(ftr, target)
print('Best parameters found by grid search are:', clf.best_params_)

### XGBOOST 모델

In [13]:
# 기존 옵션 - 떨어지면 다시 넣자(, silent= 1, subsample= 0.8, nthread= 4, colsample_bytree= 0.7, min_child_weight= 11, missing= -999,)
xgb = XGBClassifier(learning_rate= 0.05, max_depth= 6, n_estimators= 350,  objective= 'binary:logistic', seed= 1337)
xgb.fit(ftr, target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=350, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
              silent=None, subsample=1, verbosity=1)

### LGBM 최적 Parameter 찾기

In [ ]:
estimator = LGBMClassifier(num_leaves=2)
param_grid = {
    'learning_rate': [0.03, 0.04],    # 0.04
    'n_estimators': [1000,1100],    # 1100
    'max_depth' : [24,25,26]          # 25
}

grid = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc_ovr')

grid.fit(ftr, target)

print('Best parameters found by grid search are:', grid.best_params_)

### LGBM 모델

In [14]:
lgb = LGBMClassifier(n_estimators=1000,  max_depth=24, learning_rate=0.03)

# evals = [(ftr, target)] : 주석 처리한 부분은 점수를 확인하기 위한 것.

lgb.fit(ftr, target,  eval_metric='logloss', verbose=True)   #early_stopping_rounds=100,, eval_set=evals

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.03, max_depth=24,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### 앙상블 Voting

In [ ]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[('rfc', rfc), ('lgb', lgb), ('xgb', xgb)], voting='soft')
vclf.fit(ftr, target)
y_prob=eclf.predict_proba(X_pivot_test)

### 정확도 그래프 그리기

In [ ]:
 fig = plt.figure()
    # 정확도 그래프 그리기
    plt.plot(range(epochs), history['val_acc'], label='Accuracy', color='darkred')
    # 축 이름
    plt.xlabel('epochs')
    plt.ylabel('검증 정확도(%)')
    plt.title('epoch에 따른 검증 데이터에 대한 정확도 그래프')
    plt.grid(linestyle='--', color='lavender')
    # 그래프 표시
    plt.show()

-------------------------------------
### Stacking(CV)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [ ]:
nFold = 5
rf_model = RandomForestClassifier(max_depth=19, min_samples_leaf = 1, n_estimators= 1200)
lgb = LGBMClassifier(n_estimators=1000,  max_depth=24, learning_rate=0.03)
# lr_model = LogisticRegression()

rf_train_fold_predict  = np.zeros((ftr.shape[0], 1))
lgb_train_fold_predict = np.zeros((ftr.shape[0], 1))
# lr_train_fold_predict  = np.zeros((ftr.shape[0], 1))

rf_test_predict  = np.zeros((X_pivot_test.shape[0], nFold))
lgb_test_predict = np.zeros((X_pivot_test.shape[0], nFold))
# lr_test_predict  = np.zeros((X_pivot_test.shape[0], 5))

In [ ]:
print(rf_train_fold_predict.shape)
print(lgb_train_fold_predict.shape)
# print(lr_train_fold_predict.shape)
print('---------------------------')
print(rf_test_predict.shape)
print(lgb_test_predict.shape)
# print(lr_test_predict.shape)

(3125, 1)
(3125, 1)
---------------------------
(782, 5)
(782, 5)


In [ ]:
print(target.shape)
print(ftr.shape)
print(X_pivot_test.shape)

(3125,)
(3125, 43)
(782, 43)


In [ ]:
skFold = StratifiedKFold(n_splits=nFold, random_state=111, shuffle=True)
for loop_cnt, (train_fold_idx, val_fold_idx) in enumerate(skFold.split(ftr, target)):
    X_train_fold = ftr.iloc[train_fold_idx]
    y_train_fold = target.iloc[train_fold_idx]
    X_test_fold   = ftr.iloc[val_fold_idx]
    
    #--------------------------------------------------------- fit    X_train_fold, y_train_fold
    rf_model.fit(X_train_fold, y_train_fold)
    lgb.fit(X_train_fold, y_train_fold)
    # lr_model.fit(X_train_fold, y_train_fold)
    # --------------------------------------------------------- predict  X_test_fold
    rf_pred_fold = rf_model.predict(X_test_fold)
    lgb_pred_fold = lgb.predict(X_test_fold)
    # lr_pred_fold = lr_model.predict(X_test_fold)
    print(loop_cnt, rf_pred_fold.shape,  lgb_pred_fold.shape) #, lr_pred_fold.shape)

    rf_train_fold_predict[val_fold_idx , :]   = rf_pred_fold.reshape(-1, 1)
    lgb_train_fold_predict[val_fold_idx , :]  = lgb_pred_fold.reshape(-1, 1)
    # lr_train_fold_predict[val_fold_idx , :]   = lr_pred_fold.reshape(-1, 1)
    # --------------------------------------------------------- predict   X_val
    # rf_pred_val  = rf_model.predict(X_val)
    # svc_pred_val = svc_model.predict(X_val)
    # lr_pred_val  = lr_model.predict(X_val)

    # rf_test_predict[:, loop_cnt]  = rf_pred_val
    # svc_test_predict[:, loop_cnt] = svc_pred_val
    # lr_test_predict[:, loop_cnt]  = lr_pred_val

0 (625,) (625,)
1 (625,) (625,)
2 (625,) (625,)
3 (625,) (625,)
4 (625,) (625,)


In [ ]:
rf_test_predict_mean = np.mean(rf_test_predict, axis=1)
print(rf_test_predict_mean.shape)
rf_test_predict_mean = rf_test_predict_mean.reshape(-1, 1)
print(rf_test_predict_mean.shape)

lgb_test_predict_mean = np.mean(lgb_test_predict, axis=1)
print(lgb_test_predict_mean.shape)
lgb_test_predict_mean = lgb_test_predict_mean.reshape(-1, 1)
print(lgb_test_predict_mean.shape)

# lr_test_predict_mean = np.mean(lr_test_predict, axis=1)
# print(lr_test_predict_mean.shape)
# lr_test_predict_mean = lr_test_predict_mean.reshape(-1, 1)
# print(lr_test_predict_mean.shape)

(782,)
(782, 1)
(782,)
(782, 1)


In [ ]:
new_train_data = np.concatenate([rf_train_fold_predict, lgb_train_fold_predict], axis = 1) #, lr_train_fold_predict], axis=1)
new_test_mean = np.concatenate([rf_test_predict_mean, lgb_test_predict_mean], axis = 1)    #, lr_test_predict_mean], axis=1)
print(new_train_data.shape)
print(new_test_mean.shape)
print(y_train.shape)

(3125, 2)
(782, 2)
(2500,)


In [ ]:
estimator = XGBClassifier()
xgb = XGBClassifier(learning_rate= 0.05, max_depth= 6, n_estimators= 350,  objective= 'binary:logistic', seed= 1337)
xgb.fit(new_train_data, target)
xgb_prob=xgb.predict_proba(new_test_mean)

In [ ]:
xgb_prob.shape

(782, 61)

In [ ]:
submission.iloc[:,1:] = xgb_prob
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
1,3126,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
2,3127,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
3,3128,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
4,3129,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,0.000815,0.000871,0.000221,0.000212,0.000278,0.00048,0.000161,0.041314,0.000238,0.000034,0.000377,0.003896,0.000252,0.00025,0.040836,0.00012,0.009802,0.000831,0.023184,0.000232,0.000951,0.000295,0.000293,0.000115,0.000132,0.017449,0.004707,0.000175,0.000319,0.000046,0.000231
778,3903,0.002248,0.610373,0.014624,0.017738,0.005253,0.002051,0.001163,0.039656,0.001218,0.0007,0.000819,0.000834,0.000413,0.000384,0.000554,0.000425,0.002588,0.002529,0.00022,0.00047,0.000227,0.000249,0.014426,0.000985,0.000923,0.000279,0.085928,0.041238,0.002111,0.000259,

In [ ]:
submission.to_csv(d_path + 'baseline_rf_final.csv', index=False)